In [1]:
# 最高ウェイト表現のウェイトを最高ウェイトとの差で表す関数。
def root_difference_multiplicities(character_ring, highest_weight) -> dict:
    weight_muliplicities = character_ring(highest_weight).weight_multiplicities()
    A = matrix([ vector(sr) for sr in character_ring.simple_roots() ]).transpose()

    result ={}

    for k, v in weight_muliplicities.items():
        Y = vector(k) - vector(highest_weight)
        result[tuple(A.solve_right(Y))] = v
    
    return result

In [2]:
# Parabolic subgroupを表すクラス
class ParabolicSubgroup:
    def __init__(self, G, L, crossed_out_nodes) -> None:
        self.G = G
        self.L = L
        self.crossed_out_nodes = crossed_out_nodes
        self.R_G = WeylCharacterRing(self.G)
        self.R_L = WeylCharacterRing(self.L)

    def __repr__(self) -> str:
        return f'the parabolic subgroup of {self.G} with crossed-out nodes {self.crossed_out_nodes}'

    def dynkin_diagram(self):
        return self.G.marked_nodes(self.crossed_out_nodes).dynkin_diagram()

    def ambient_space(self):
        return self.R_G.space()
    
    def ambient_space_dimension(self):
        return self.R_G.space().dimension()

    # Sagemathのライブラリでは1-indexを用いるが、ここでは簡単のためとりあえず0-indexで表す
    def simple_roots(self):
        return [ self.R_G.simple_roots()[i] for i in set(1..self.G.rank()) - set(self.crossed_out_nodes)]

    def positive_roots(self):
        L = self.ambient_space()

        # positive rootからcutoutされるsimple rootを引き, positiveでなければuncrossed nodeから生成されると判定
        roots = [ pr for pr in self.G.root_system().root_lattice().positive_roots()]
        for i in self.crossed_out_nodes:
            roots = [ pr for pr in roots if not (pr - (self.G.root_system().root_lattice().simple_roots())[i]).is_positive_root()]

        # ambient spaceの元に変換
        return [ L(pr) for pr in roots ]

    # 引数の`weight`は基本ウェイトを基底にして表示したもの
    def weight_muliplicities(self, weight) -> dict:
        # GとLのディンキン図の頂点のずれを補正する関数
        def correct_index(index: int) -> int:
            for i in range(len(self.crossed_out_nodes)):
                if index + i < self.crossed_out_nodes[i]:
                    return index + i
            return index + len(self.crossed_out_nodes)

        fws_L = [fw for fw in self.R_L.fundamental_weights()] # conversion from 1-index to 0-index
        weight_for_L = [ weight[i - 1] for i in set(1..len(weight))-set(self.crossed_out_nodes)] 
        weight_for_L = sum( weight_for_L[i] * fws_L[i] for i in range(self.L.rank()))

        fws_G = [fw for fw in self.R_G.fundamental_weights()]
        weight_for_G = sum( weight[i] * fws_G[i] for i in range(self.G.rank()))

        mul_set = root_difference_multiplicities(self.R_L, weight_for_L)

        result = {}
        for k, v in mul_set.items():
            w = weight_for_G \
                + sum(k[i - 1] * self.R_G.simple_roots()[correct_index(i)] 
                    for i in (1..self.L.rank()))
            result[w] = v

        return result
    
    def weyl_dimension(self, weight) -> int:
        return self.ambient_space().weyl_dimension(sum(weight[i - 1] * self.R_G.fundamental_weights()[i] for i in (1..self.G.rank())))

In [3]:
P = ParabolicSubgroup(CartanType('A3'), CartanType('A2'), [1])
P

the parabolic subgroup of ['A', 3] with crossed-out nodes [1]

In [4]:
P.dynkin_diagram()

X---O---O
1   2   3   
A3 with node 1 marked

In [5]:
P.ambient_space_dimension()

4

In [6]:
P.simple_roots()

[(0, 1, -1, 0), (0, 0, 1, -1)]

In [7]:
P.positive_roots()

[(0, 1, -1, 0), (0, 0, 1, -1), (0, 1, 0, -1)]

In [8]:
P.weight_muliplicities((1, 2, 0))

{(3, 2, 0, 0): 1,
 (3, 1, 1, 0): 1,
 (3, 0, 2, 0): 1,
 (3, 1, 0, 1): 1,
 (3, 0, 1, 1): 1,
 (3, 0, 0, 2): 1}

In [9]:
from abc import ABC, abstractmethod

class IVectorBundle(ABC):
    pass

class IVariety(ABC):
    @abstractmethod
    def dimension(self) -> int:
        pass

    @abstractmethod
    def tangent_bundle(self) -> IVectorBundle:
        pass

    def cotangent_bundle(self) -> IVectorBundle:
        return self.tangent_bundle().dual()

    @abstractmethod
    def integration(self, f) -> int:
        pass

    def chern_classes(self) -> list:
        return self.tangent_bundle().cheern_classes()

    def todd_classes(self) -> list:
        return self.tangent_bundle().todd_classes()

In [10]:
import re
singular.lib("chern.lib")


class IVectorBundle(ABC):
    @abstractmethod
    def base(self) -> IVariety:
        pass

    @abstractmethod
    def rank(self) -> int:
        pass

    @abstractmethod
    def chern_classes(self) -> list:
        pass

    # 演算子のオーバーロード
    def __add__(self, other) -> IVectorBundle:
        return direct_sum(self, other)

    def __mul__(self, other) -> IVectorBundle:
        return tensor_product(self, other)

    def chern_character(self) -> list:
        len_cc = len(self.chern_classes()) - 1

        ring_for_ch = PolynomialRing(QQ, [ f'c{i}_E' for i in (1..len_cc) ], order=TermOrder('wdeglex', tuple(1..len_cc)))
        
        # Using Singular, calclate universal formula of chern character
        singular.lib('chern.lib')
        r = singular.ring(0, f'(c(1..{len_cc}))', 'dp')
        l = singular.list(f'c(1..{len_cc})')
        ch_str_list = singular.chAll(l, self.base().dimension()).sage_structured_str_list()
        chern_character = [ ring_for_ch(self.rank()) ] + [ ring_for_ch(re.sub(r'c\(([0-9]+)\)', r'c\1_E', s)) for s in ch_str_list ]

        chern_classes = self.chern_classes()[1:]

        return [ chern_character[i](chern_classes) for i in (0..self.base().dimension()) ]

    def todd_classes(self) -> list:
        len_cc = len(self.chern_classes()) - 1

        ring_for_td = PolynomialRing(QQ, [ f'c{i}_M' for i in (1..len_cc) ], order=TermOrder('wdeglex', tuple(1..len_cc)))

        # Using Singular, calculate universal formula of Todd classes
        r = singular.ring(0, f'(c(1..{len_cc}))', 'dp')
        l = singular.list(f'c(1..{len_cc})')
        todd_str_list = singular.todd(l).sage_structured_str_list()
        todd_classes = [ ring_for_td(1) ] + [ ring_for_td(re.sub(r'c\(([0-9]+)\)', r'c\1_M', s)) for s in todd_str_list ]

        chern_classes = self.chern_classes()[1:]

        return [ todd_classes[i](chern_classes) for i in (0..self.base().dimension()) ]

    def dual(self) -> IVectorBundle:
        vector_bundle = self

        class VB(IVectorBundle):
            def rank(self) -> int:
                return vector_bundle.rank()
            
            def base(self) -> IVariety:
                return vector_bundle.base()

            def chern_classes(self) -> list:
                chern_classes = vector_bundle.chern_classes()
                return [ (-1)^i * chern_classes[i] for i in range(len(chern_classes))]

            def __repr__(self) -> str:
                return f'the dual vector bundle of {vector_bundle}'

        return VB()

In [11]:
def direct_sum(vector_bundle1: IVectorBundle, vector_bundle2: IVectorBundle):
    if vector_bundle1.base() != vector_bundle2.base():
        raise TypeError('Not match bases of vector bundles')
    
    
    rank = vector_bundle1.rank() + vector_bundle2.rank()
    base = vector_bundle1.base()

    cc = sum(c1 for c1 in vector_bundle1.chern_classes()) * sum(c2 for c2 in vector_bundle2.chern_classes())
    chern_classes = [
        homogeneous_part(cc, i)
        for i in (0..vector_bundle1.base().dimension())
    ]

    class VB(IVectorBundle):
        def rank(self) -> int:
            return rank
        
        def base(self) -> IVariety:
            return base

        def chern_classes(self) -> list:
            return chern_classes

        def __repr__(self) -> str:
            return f'the direct sum of {vector_bundle1} and {vector_bundle2}'

    return VB()

In [12]:
def tensor_product(vector_bundle1: IVectorBundle, vector_bundle2: IVectorBundle):
    if vector_bundle1.base() != vector_bundle2.base():
        raise TypeError('Not match bases of vector bundles')
    
    rank1 = vector_bundle1.rank()
    rank2 = vector_bundle2.rank()

    len_cc1 = len(vector_bundle1.chern_classes()) - 1
    len_cc2 = len(vector_bundle2.chern_classes()) - 1
    
    ring_for_Es = PolynomialRing(QQ, [ f'c{i}_E1' for i in (1..len_cc1) ] + [ f'c{i}_E2' for i in (1..len_cc2) ])
    

    r = singular.ring(0, f'(c(1..{len_cc1}), C(1..{len_cc2}))', 'dp')
    l1 = singular.list(f'c(1..{len_cc1})')
    l2 = singular.list(f'C(1..{len_cc2})')

    ch_str_list = singular.chProd(rank1, l1, rank2, l2).sage_structured_str_list()
    ch_prod = [ re.sub(r'c\(([0-9]+)\)', r'c\1_E1', s) for s in ch_str_list[:vector_bundle1.base().dimension()] ]
    ch_prod = [ ring_for_Es(re.sub(r'C\(([0-9]+)\)', r'c\1_E2', s)) for s in ch_prod ]


    cc = [ 1 ] + [ cp(vector_bundle1.chern_classes()[1:] + vector_bundle2.chern_classes()[1:]) for cp in ch_prod ]
    

    rank = vector_bundle1.rank() * vector_bundle2.rank()
    base = vector_bundle1.base()


    class VB(IVectorBundle):
        def rank(self) -> int:
            return rank
        
        def base(self) -> IVariety:
            return base

        def chern_classes(self) -> list:
            return cc

        def __repr__(self) -> str:
            return f'the tensor product of {vector_bundle1} and {vector_bundle2}'

    return VB()

In [13]:
# `degree`次部分を取り出す関数
homogeneous_part = lambda F, degree: sum( c*m for c, m in F if m.total_degree() == degree )

In [14]:
class FlagVariety(IVariety):
    def __init__(self, parabolic_subgroup) -> None:
        self.parabolic_subgroup = parabolic_subgroup

        # コホモロジー環を含む環
        self.ring = PolynomialRing(QQ, 'x', parabolic_subgroup.ambient_space_dimension()) 
        self.x = self.ring.gens()

        self.tangent_weights = [
            sum( r[l] * self.x[l] for l in range(parabolic_subgroup.ambient_space_dimension())) 
            for r in set(parabolic_subgroup.R_G.positive_roots())-set(parabolic_subgroup.positive_roots())
        ]
        self.dim = len(self.tangent_weights)

    def __repr__(self) -> str:
        return f'a flag_variety associated to {self.parabolic_subgroup}'

    def dimension(self) -> int:
        return self.dim
    
    def tangent_bundle(self):
        tangent_weights = { w: 1 for w in set(self.parabolic_subgroup.R_G.positive_roots())-set(self.parabolic_subgroup.positive_roots()) }
        return EquivariantVectorBundle(self, tangent_weights)


    # 次数ごとのchern類
    def chern_classes(self):
        return [ 
            homogeneous_part(prod(1+x for x in self.tangent_weights), i)
            for i in (0..self.dim) 
        ]

    def numerical_integration_by_localization(self, f):
        random_x = [RealField(1000)(random()) for i in range(self.parabolic_subgroup.ambient_space_dimension())]
        orbit_of_random_x = [(w.inverse()*vector(RealField(1000),random_x)).list() for w in WeylGroup(self.parabolic_subgroup.G)]
        top_of_f = homogeneous_part(f, self.dim)
        denominator_in_localization = prod(self.tangent_weights)
        return sum([top_of_f(x)/denominator_in_localization(x) for x in orbit_of_random_x]).round() / len(WeylGroup(self.parabolic_subgroup.L))

    def integration(self, f) -> int:
        return self.numerical_integration_by_localization(f)
     

In [15]:
class EquivariantVectorBundle(IVectorBundle):
    def __init__(self, flag_variety, weight_muliplicities) -> None:
        self.flag_variety = flag_variety
        self.weight_muliplicities = weight_muliplicities
        self.rk = sum(v for v in self.weight_muliplicities.values())

    def __repr__(self) -> str:
        return f'an equivariant vector bundle on {self.flag_variety} associated to {self.weight_muliplicities}'

    def rank(self) -> int:
        return self.rk

    def base(self) -> FlagVariety:
        return self.flag_variety

    def chern_classes(self):
        def class_from_weight(weight):
            return sum(weight[i] * self.flag_variety.x[i] for i in range(self.flag_variety.parabolic_subgroup.ambient_space_dimension()))
        
        cc = prod((1 + class_from_weight(vector(w)))^i for w, i in self.weight_muliplicities.items())

        return [
            homogeneous_part(cc, i)
            for i in (0..self.flag_variety.dim)
        ]

class IrreducibleEquivariantVectorBundle(EquivariantVectorBundle):
    def __init__(self, flag_variety, weight) -> None:
        self.weight = weight
        super().__init__(flag_variety, flag_variety.parabolic_subgroup.weight_muliplicities(weight))

    def __repr__(self) -> str:
        return f'an equivariant vector bundle on {self.flag_variety} associated to {self.weight}'

- Test of FlagVariety

In [16]:
X = FlagVariety(P)
X

a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1]

In [17]:
X.dimension()

3

In [18]:
X.tangent_weights

[x0 - x2, x0 - x1, x0 - x3]

In [19]:
X.chern_classes()

[1,
 3*x0 - x1 - x2 - x3,
 3*x0^2 - 2*x0*x1 - 2*x0*x2 + x1*x2 - 2*x0*x3 + x1*x3 + x2*x3,
 x0^3 - x0^2*x1 - x0^2*x2 + x0*x1*x2 - x0^2*x3 + x0*x1*x3 + x0*x2*x3 - x1*x2*x3]

In [20]:
X.todd_classes()

[1,
 3/2*x0 - 1/2*x1 - 1/2*x2 - 1/2*x3,
 x0^2 - 2/3*x0*x1 + 1/12*x1^2 - 2/3*x0*x2 + 1/4*x1*x2 + 1/12*x2^2 - 2/3*x0*x3 + 1/4*x1*x3 + 1/4*x2*x3 + 1/12*x3^2,
 3/8*x0^3 - 3/8*x0^2*x1 + 1/12*x0*x1^2 - 3/8*x0^2*x2 + 7/24*x0*x1*x2 - 1/24*x1^2*x2 + 1/12*x0*x2^2 - 1/24*x1*x2^2 - 3/8*x0^2*x3 + 7/24*x0*x1*x3 - 1/24*x1^2*x3 + 7/24*x0*x2*x3 - 1/8*x1*x2*x3 - 1/24*x2^2*x3 + 1/12*x0*x3^2 - 1/24*x1*x3^2 - 1/24*x2*x3^2]

In [21]:
X.tangent_bundle()

an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1] associated to {(1, 0, -1, 0): 1, (1, -1, 0, 0): 1, (1, 0, 0, -1): 1}

In [22]:
X.cotangent_bundle().chern_classes()

[1,
 -3*x0 + x1 + x2 + x3,
 3*x0^2 - 2*x0*x1 - 2*x0*x2 + x1*x2 - 2*x0*x3 + x1*x3 + x2*x3,
 -x0^3 + x0^2*x1 + x0^2*x2 - x0*x1*x2 + x0^2*x3 - x0*x1*x3 - x0*x2*x3 + x1*x2*x3]

- Test for EquivariantVectorBundle

In [23]:
w = (4, 0, 0)
E = IrreducibleEquivariantVectorBundle(X, w)
E

an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1] associated to (4, 0, 0)

In [24]:
E.base()

a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1]

In [25]:
E.rank()

1

In [26]:
E.chern_classes()

[1, 4*x0, 0, 0]

In [27]:
E.chern_character()

[1, 4*x0, 8*x0^2, 32/3*x0^3]

In [28]:
E.todd_classes()

[1, 2*x0, 4/3*x0^2, 0]

In [29]:
class CompleteIntersection(IVariety):
    def __init__(self, flag_variety, vector_bundle) -> None:
        self.flag_variety = flag_variety
        self.vector_bundle = vector_bundle
        self.dim = self.flag_variety.dimension() - self.vector_bundle.rank()

    def __repr__(self) -> str:
        return f'a complete intersection of {self.flag_variety} and {self.vector_bundle}'

    def dimension(self) -> int:
        return self.dim
    
    def tangent_bundle(self):
        ci = self
        class VB(IVectorBundle):
            def base(self) -> IVariety:
                return ci
            def rank(self) -> int:
                return ci.dim
            def chern_classes(self) -> list:
                return ci.chern_classes()
        
        return VB()

    def chern_classes(self):
        def class_from_weight(weight):
            return sum(weight[i] * self.flag_variety.x[i] for i in range(self.flag_variety.parabolic_subgroup.ambient_space_dimension()))
        
        def geometric_sequence(n, x):
            return sum( x^i for i in (0..n))
        
        cc = prod(1+x for x in self.flag_variety.tangent_weights) \
            * \
            prod(geometric_sequence(self.dim, -class_from_weight(vector(w)))^i 
                for w, i in self.vector_bundle.weight_muliplicities.items()
            )
        
        return [
            homogeneous_part(cc, i)
            for i in (0..self.dim)
        ]

    def numerical_integration_by_localization(self, f):
        top_of_f = homogeneous_part(f, self.dim)
        c_top    = self.vector_bundle.chern_classes()[self.vector_bundle.rank()] if self.dim >= 0 else 0
        return self.flag_variety.numerical_integration_by_localization(top_of_f * c_top)

    def integration(self, f) -> int:
        return self.numerical_integration_by_localization(f)


In [30]:
# K3
Y = CompleteIntersection(X, E) 
Y.dimension()

2

In [31]:
Y.chern_classes()

[1,
 -x0 - x1 - x2 - x3,
 7*x0^2 + 2*x0*x1 + 2*x0*x2 + x1*x2 + 2*x0*x3 + x1*x3 + x2*x3]

In [32]:
Y.tangent_bundle().chern_classes()

[1,
 -x0 - x1 - x2 - x3,
 7*x0^2 + 2*x0*x1 + 2*x0*x2 + x1*x2 + 2*x0*x3 + x1*x3 + x2*x3]

In [33]:
Y.cotangent_bundle().chern_classes()

[1,
 x0 + x1 + x2 + x3,
 7*x0^2 + 2*x0*x1 + 2*x0*x2 + x1*x2 + 2*x0*x3 + x1*x3 + x2*x3]

In [34]:
Y.integration(Y.chern_classes()[Y.dimension()])

24

Euler Characteristic
$$
\chi (X, E) = \int_X ch(E) td(X)
$$

In [35]:
def euler_characteristic(variety: IVariety, vector_bundle: IVectorBundle) -> int:
    chern_character = vector_bundle.chern_character()
    todd_classes    = variety.todd_classes()
    
    return variety.integration( 
        sum(
            chern_character[i]
            * todd_classes[variety.dimension() - i]
            for i in (0..variety.dimension())
        )
    )

In [36]:
euler_characteristic(X, E)

35

In [37]:
P.weyl_dimension(w)

35

- Tests for 
    - dual bundle
    - direct sum
    - tensor product
    - Symmetric products
    - Wedge products

In [38]:
E = IrreducibleEquivariantVectorBundle(X, (1, 1, 0))
E.rank()

3

In [39]:
dual_of_E = E.dual()
dual_of_E

the dual vector bundle of an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1] associated to (1, 1, 0)

In [40]:
dual_of_E.rank()

3

In [41]:
E.chern_classes()

[1,
 6*x0 + x1 + x2 + x3,
 12*x0^2 + 4*x0*x1 + 4*x0*x2 + x1*x2 + 4*x0*x3 + x1*x3 + x2*x3,
 8*x0^3 + 4*x0^2*x1 + 4*x0^2*x2 + 2*x0*x1*x2 + 4*x0^2*x3 + 2*x0*x1*x3 + 2*x0*x2*x3 + x1*x2*x3]

In [42]:
dual_of_E.chern_classes()

[1,
 -6*x0 - x1 - x2 - x3,
 12*x0^2 + 4*x0*x1 + 4*x0*x2 + x1*x2 + 4*x0*x3 + x1*x3 + x2*x3,
 -8*x0^3 - 4*x0^2*x1 - 4*x0^2*x2 - 2*x0*x1*x2 - 4*x0^2*x3 - 2*x0*x1*x3 - 2*x0*x2*x3 - x1*x2*x3]

In [43]:
L1 = IrreducibleEquivariantVectorBundle(X, (2, 0, 0))
L2 = IrreducibleEquivariantVectorBundle(X, (5, 0, 0))
L3 = IrreducibleEquivariantVectorBundle(X, (7, 0, 0))

In [44]:
direct_sum(L1, L2)

the direct sum of an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1] associated to (2, 0, 0) and an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1] associated to (5, 0, 0)

In [45]:
direct_sum(L1, L2).rank()

2

In [46]:
direct_sum(L1, L2).chern_classes()

[1, 7*x0, 10*x0^2, 0]

In [47]:
(L1 + L2 + L3).chern_classes()

[1, 14*x0, 59*x0^2, 70*x0^3]

In [48]:
E_otimes_L1 = tensor_product(E, L1)
E_otimes_L1

the tensor product of an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1] associated to (1, 1, 0) and an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1] associated to (2, 0, 0)

In [49]:
(L1 * L2 * L3).chern_classes()

[1, 14*x0]

In [50]:
def symmetric_product(vector_bundle: IVectorBundle, k: int) -> IVectorBundle:
    chern_classes = vector_bundle.chern_classes()[1:]
    rank          = vector_bundle.rank()
    len_cc        = len(chern_classes)

    ring_for_E = PolynomialRing(QQ, [ f'c{i}_E' for i in (1..len_cc) ], order=TermOrder('wdeglex', tuple(1..len_cc)))

    r = singular.ring(0, f'(c(1..{len_cc}))', 'dp')
    l = singular.list(f'c(1..{len_cc})')
    ch_symm_str_list = singular.chSymm(k, rank, l).sage_structured_str_list()
    ch_symm = [ ring_for_E(re.sub(r'c\(([0-9]+)\)', r'c\1_E', s)) for s in ch_symm_str_list[1][:vector_bundle.base().dimension()] ]

    rank = int(ch_symm_str_list[0])
    base = vector_bundle.base()
    cc = [ 1 ] + [ cs(chern_classes) for cs in ch_symm ]

    class VB(IVectorBundle):
        def rank(self) -> int:
            return rank
        
        def base(self) -> IVariety:
            return base
        
        def chern_classes(self) -> list:
            return cc

        def __repr__(self) -> str:
            return f'the {k}-th symmetric product of {vector_bundle}'

    return VB()

In [51]:
S3_of_E = symmetric_product(E, 3)
S3_of_E.rank()

10

In [52]:
S3_of_E.chern_classes()

[1,
 60*x0 + 10*x1 + 10*x2 + 10*x3,
 1620*x0^2 + 540*x0*x1 + 40*x1^2 + 540*x0*x2 + 95*x1*x2 + 40*x2^2 + 540*x0*x3 + 95*x1*x3 + 95*x2*x3 + 40*x3^2,
 25920*x0^3 + 12960*x0^2*x1 + 1920*x0*x1^2 + 82*x1^3 + 12960*x0^2*x2 + 4560*x0*x1*x2 + 357*x1^2*x2 + 1920*x0*x2^2 + 357*x1*x2^2 + 82*x2^3 + 12960*x0^2*x3 + 4560*x0*x1*x3 + 357*x1^2*x3 + 4560*x0*x2*x3 + 852*x1*x2*x3 + 357*x2^2*x3 + 1920*x0*x3^2 + 357*x1*x3^2 + 357*x2*x3^2 + 82*x3^3]

In [53]:
def wedge_product(vector_bundle: IVectorBundle, k: int) -> IVectorBundle:
    chern_classes = vector_bundle.chern_classes()[1:]
    rank          = vector_bundle.rank()
    len_cc        = len(chern_classes)

    ring_for_E = PolynomialRing(QQ, [ f'c{i}_E' for i in (1..len_cc) ], order=TermOrder('wdeglex', tuple(1..len_cc)))

    r = singular.ring(0, f'(c(1..{len_cc}))', 'dp')
    l = singular.list(f'c(1..{len_cc})')
    ch_wedge_str_list = singular.chWedge(k, rank, l).sage_structured_str_list()
    ch_wedge = [ ring_for_E(re.sub(r'c\(([0-9]+)\)', r'c\1_E', s)) for s in ch_wedge_str_list[1][:vector_bundle.base().dimension()] ]

    rank = int(ch_wedge_str_list[0])
    base = vector_bundle.base()
    cc   = [ 1 ] + [ cs(chern_classes) for cs in ch_wedge ]

    class VB(IVectorBundle):
        def rank(self) -> int:
            return rank
        
        def base(self) -> IVariety:
            return base
        
        def chern_classes(self) -> list:
            return cc

        def __repr__(self) -> str:
            return f'the {k}-th wedge product of {vector_bundle}'

    return VB()

In [54]:
L2_of_E = wedge_product(E, 2)
L2_of_E

the 2-th wedge product of an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1] associated to (1, 1, 0)

In [55]:
L2_of_E.rank()

3

In [56]:
L2_of_E.chern_classes()

[1,
 12*x0 + 2*x1 + 2*x2 + 2*x3,
 48*x0^2 + 16*x0*x1 + x1^2 + 16*x0*x2 + 3*x1*x2 + x2^2 + 16*x0*x3 + 3*x1*x3 + 3*x2*x3 + x3^2,
 64*x0^3 + 32*x0^2*x1 + 4*x0*x1^2 + 32*x0^2*x2 + 12*x0*x1*x2 + x1^2*x2 + 4*x0*x2^2 + x1*x2^2 + 32*x0^2*x3 + 12*x0*x1*x3 + x1^2*x3 + 12*x0*x2*x3 + 2*x1*x2*x3 + x2^2*x3 + 4*x0*x3^2 + x1*x3^2 + x2*x3^2]

In [57]:
wedge_product(E, 3).rank()

1

In [58]:
wedge_product(E, 5).rank()

0

$\chi_y$-Genus
$$
\chi_y(X) = \int_X ch(\Lambda_y T^{\vee}_X) td(X)
$$

In [59]:
R = QQ['y']
y = R.gen()

def chi_y(variety: IVariety):
    cotangent_bundle = variety.cotangent_bundle()
    todd_classes = variety.todd_classes()

    coeff = [ 0 ] * (variety.dimension() + 1)

    for k in (0..variety.dimension()):
        chern_character = wedge_product(cotangent_bundle, k).chern_character()
        coeff[k] = variety.integration( 
            sum(
                chern_character[i]
                * todd_classes[variety.dimension() - i]
                for i in (0..variety.dimension())
            )
        )

    return sum(coeff[i] * y^i for i in (0..variety.dimension()))

In [60]:
chi_y(Y)

2*y^2 - 20*y + 2

In [61]:
chi_y(Y)(-1) # オイラー標数と同じ

24

======= F4 cases =======

In [62]:
G = CartanType('F4')
G.dynkin_diagram()

O---O=>=O---O
1   2   3   4   
F4

In [63]:
P1 = ParabolicSubgroup(G, CartanType('A1', 'A2'), [2])
P1

the parabolic subgroup of ['F', 4] with crossed-out nodes [2]

In [64]:
P1.dynkin_diagram()

O---X=>=O---O
1   2   3   4   
F4 with node 2 marked

In [65]:
X1 = FlagVariety(P1)
X1

a flag_variety associated to the parabolic subgroup of ['F', 4] with crossed-out nodes [2]

In [66]:
w1 = (0, 1, 1, 0)
E1 = IrreducibleEquivariantVectorBundle(X1, w1)
E1

an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['F', 4] with crossed-out nodes [2] associated to (0, 1, 1, 0)

In [67]:
E1.rank()

3

In [68]:
euler_characteristic(X1, E1)

107406

In [69]:
P1.weyl_dimension(w1)

107406

In [70]:
w2 = (5, 0, 0, 0)
E2 = IrreducibleEquivariantVectorBundle(X1, w2)
E2.rank()

6

In [71]:
euler_characteristic(X1, E2)

627912

In [72]:
P1.weyl_dimension(w2)

627912

In [73]:
P2 = ParabolicSubgroup(G, CartanType('A2', 'A1'), [3])
P2

the parabolic subgroup of ['F', 4] with crossed-out nodes [3]

In [74]:
P2.dynkin_diagram()

O---O=>=X---O
1   2   3   4   
F4 with node 3 marked

In [75]:
X2 = FlagVariety(P2)
X2

a flag_variety associated to the parabolic subgroup of ['F', 4] with crossed-out nodes [3]

In [76]:
w3 = (0, 1, 1, 0)
E3 = IrreducibleEquivariantVectorBundle(X2, w3)
E3

an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['F', 4] with crossed-out nodes [3] associated to (0, 1, 1, 0)

In [77]:
euler_characteristic(X2, E3)

107406

In [78]:
P2.weyl_dimension(w3)

107406

In [79]:
w4 = (2, 1, 0, 0)
E4 = IrreducibleEquivariantVectorBundle(X2, w4)
E4.rank()

15

In [80]:
euler_characteristic(X2, E4)

340119

In [81]:
P2.weyl_dimension(w4)

340119